# Урок 14. Transfer learning
## Задание

1. Взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes. Обучить модель GPT для генерации своих цитат

2. Взять новостные данные из https://github.com/natasha/corus load_lenta2. Нам понадобиться сам текст и заголовок. Обучить модель T5/ или GPT для генерации заголовков для статей

## Задание 1.

In [1]:
import json
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

In [2]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

Загрузим датасет:

In [3]:
data = []
with open('../data/dataset.jsonl', 'r', encoding='utf-8') as json_file:
    json_list = list(json_file)
for json_str in json_list:
    data.append(json.loads(json_str)["text"])

In [4]:
data[:5]

['<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)',
 '<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/',
 '<Дор> "мышка, почему у тебя такие большие глаза?" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!',
 '<PPDV[os2]> "Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам"',
 '<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.']

In [5]:
len(data)

81497

Отберем 5000 цитат:

In [6]:
data = data[:5000]

In [7]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w', encoding="utf-8")
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        # summary = re.sub(r'<.*?>', " ", summary) # Убираем никнеймы
        summary = re.sub(r"\s", " ", summary) 
        data += summary + "  "
    f.write(data)

In [8]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [9]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 4250
Test dataset length: 750


In [10]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

C:\Users\Armik\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [12]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [13]:
training_args = TrainingArguments(
    output_dir="./gpt_bash", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Обучим GPT:

In [15]:
trainer.train()

C:\Users\Armik\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,4.163700
1000,3.744100
1500,3.342000


TrainOutput(global_step=1635, training_loss=3.7102323666260513, metrics={'train_runtime': 7168.3957, 'train_samples_per_second': 0.912, 'train_steps_per_second': 0.228, 'total_flos': 426820534272000.0, 'train_loss': 3.7102323666260513, 'epoch': 3.0})

In [16]:
trainer.save_model()

In [17]:
tokenizer.save_pretrained('gpt_bash')
model.save_pretrained('model_gpt_bash')

In [18]:
tokenizer = AutoTokenizer.from_pretrained("gpt_bash")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_bash")

In [19]:
prefix = "Я хочу услышать лишь три слова " # "Инкапсуляция, наследование, полиморфизм"

In [20]:
def generate(prefix, gen_legth=50):
    
    tokens = tokenizer(prefix, return_tensors='pt')

    size = tokens['input_ids'].shape[1]
    output = model1.generate(
        **tokens, 
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+gen_legth, 
        repetition_penalty=5., 
        temperature=0.5,
        num_beams=10,
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    print(prefix + result)

In [21]:
generate(prefix)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Я хочу услышать лишь три слова  <REal_SM[away]> я не знаю, что ты имеешь в виду под словом "сделай мне минет", но у меня такое чувство, что это слово имеет отношение к чему-то очень значимому для


In [22]:
generate('Что делать, когда упал сервер?', gen_legth=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Что делать, когда упал сервер?  <REal_SM[away]> я не знаю как правильно пишется "сделано в России", но мне почему-то кажется, что правильнее будет сказать - сделано в Китае.  <Darth_Blade> у меня есть знакомый программист <Darth_Blade> он из Питера <Darth_Blade> работает в крупной консалтинговой фирме <Darth_Blade> и очень доволен своей работой <Darth_Blade>


Проверим цитаты на предмет - есть ли такие же цитаты в датасете:

In [26]:
def find_sentence(filename, sentence):
    with open(filename, encoding='utf-8') as file:
        for line in file:
            stringA = line
    match = re.search(sentence, stringA)

    if match:
        print('Yes!')
    else:
        print('No!')

In [27]:
find_sentence('train_dataset.txt', 'Вставай и иди на кухню пить чай с печеньем.')
find_sentence('test_dataset.txt', 'Вставай и иди на кухню пить чай с печеньем')
find_sentence('train_dataset.txt', '<REal_SM[techsupport]>')
find_sentence('train_dataset.txt', "Протестовать будем методом отключения мобильных телефонов на максимально длительный период.")
# Предыдущая фраза есть в трейновом датасете.
find_sentence('test_dataset.txt', "Протестовать будем методом отключения мобильных телефонов на максимально длительный период.")

No!
No!
No!
Yes!
No!


Как видно, часть сгенерированной фразы нигде не встречается в датасете. Это отлично.

Но в качестве разделителя предложений генератор использует никнейм. Причем, как можно видеть, никнейм выглядит, как вполне реальный, но в датасете такого не встречалось, то есть генератор его сгенерировал.


## Задание 2. Генерация заголовков

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

Загрузим данные:

In [35]:
path = '../data/lenta-ru-partial.csv'


with open(path, encoding='utf-8') as f:
    records = pd.read_csv(f)



In [59]:
records = records[:5000]

In [64]:
train, test = train_test_split(records, test_size=0.15)

In [65]:
def len_tok(text):
    return len(text.split())

In [66]:
max_len_txt, max_len_tl = max(map(len_tok, train['text'])), max(map(len_tok, train['title']))
max_len_txt, max_len_tl

(667, 14)

In [67]:
max_len_txt, max_len_tl = 1137, 15

In [68]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [69]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Функции для токенизации:

In [70]:
def tokenize(batch):
    tokenized_input = tokenizer(str(batch['text']), padding='max_length', truncation=True, max_length=max_len_txt)
    tokenized_label = tokenizer(str(batch['title']), padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']
    for key in tokenized_input:
        tokenized_input[key] = np.array(tokenized_input[key])

    return tokenized_input

In [71]:
def get_tokens(corpus, batch_size=1):
    num_batches = len(corpus) // batch_size
    last_batch = False
    if len(corpus) % batch_size > 0:
        last_batch = True
    tokens = []
    for idx in tqdm_notebook(range(num_batches)):
        token = tokenize(corpus.iloc[idx * batch_size:(idx + 1) * batch_size])
        tokens.append(token)
    if last_batch:
        tokens.append(tokenize(corpus.iloc[num_batches * batch_size:]))
    return tokens

Токенизируем датасет:

In [72]:
tokens_train = get_tokens(train)

C:\Users\Armik\AppData\Local\Temp\ipykernel_7048\826317304.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(num_batches)):


  0%|          | 0/4250 [00:00<?, ?it/s]

In [73]:
tokens_test = get_tokens(test)

C:\Users\Armik\AppData\Local\Temp\ipykernel_7048\826317304.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(num_batches)):


  0%|          | 0/750 [00:00<?, ?it/s]

In [74]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [75]:
output_dir = 'lenta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.00001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_lenta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

Обучим модель:

In [76]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokens_train,
    eval_dataset=tokens_test
)

trainer.train()

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 496423296 bytes.

In [77]:
trainer.save_model(output_dir + '/model')

In [78]:
device = "cuda"

Функция для проверки генерации заголовков:

In [81]:
import torch

def gen_title(idx):

    input_text = test.iloc[idx]['text']

    with torch.no_grad():
        tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

        source_ids = tokenized_text['input_ids'].to(dtype = torch.long)
        source_mask = tokenized_text['attention_mask'].to(dtype = torch.long)

        generated_ids = model.generate(
            input_ids = source_ids,
            attention_mask = source_mask, 
            max_length=512,
            num_beams=15,
            temperature = 0.7,
            repetition_penalty=5., 
            length_penalty=1, 
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    print("TEXT: | {}".format(test.iloc[idx]['text']))
    print("TITLE: | {}".format(test.iloc[idx]['title']))
    print("\noutput:\n" + pred)
    print('\n')

In [82]:
gen_title(2)
gen_title(3)
gen_title(4)
gen_title(5)

TEXT: | Правительство США выкупит у местных банков "плохие" активы на полтриллиона долларов. Об этом сообщает AFP со ссылкой на заявление главы министерства финансов страны Тимоти Гайтнера, объявившего о новом плане поддержки банков поздно вечером 22 марта. По словам Гайтнера, правительство не сможет справиться с кризисом в одиночку и не хочет брать на себя все риски. Поэтому американские чиновники планируют тесно работать с рынком, выкупая активы в рамках частно-государственных партнерств. Гайтнер также отметил, что в будущем программа может быть увеличена до триллиона долларов, отмечает The Wall Street Journal. В феврале уже сообщалось, что правительство США намерено создать так называемый "плохой банк", который будет собирать неликвидные активы американских финансовых организаций. На создание такого банка чиновники планировали потратить до триллиона долларов, однако детали программы не сообщались. Привлечь частных клиентов к "плохим" активам планируется за счет потенциально выгодных

**Вывод:**

Отличная генерация заголовков! Связные предложения, отражают суть статьи, по смыслу похожи на реальный заголовок, и при этом ощущается современная особенность заголовков у всех изданий - небольшой кликбейт, призванный заинтересовать пользователя

**Training was interrupted due to lack of memory. Even undertrained model works quite well**